# 라이브러리

In [1]:
import os

import pandas as pd

# 파일 불러오기

In [9]:
DATA_PATH = "./data/register_car/"
EXCEL_LIST = os.listdir(DATA_PATH)

EXCEL_LIST

['자동차등록현황보고_자동차등록대수현황 시도별 202505.xlsx', '자동차등록현황보고_자동차등록대수현황 시도별 202506.xlsx']

# 데이터 프레임으로 확인

In [12]:
df = pd.read_excel(DATA_PATH + EXCEL_LIST[0])

df

c:\khs\github\SKNW10-1st_project\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,자동차등록현황보고(Total Registered Motor Vehicles) / 자동차등록대수현황 시도별 (201101 ~ 202506),Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22
0,다운로드 시간 : 2025-08-01 09:56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,검색기간 : 202505 ~ 202505,검색분야 : 종합일반,작성주기 : 매월,단위 : 대,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,월(Monthly),시도명,시군구,승용,승용,승용,승용,승합,승합,승합,...,화물,화물,특수,특수,특수,특수,총계,총계,총계,총계
4,월(Monthly),시도명,시군구,관용,자가용,영업용,계,관용,자가용,영업용,...,영업용,계,관용,자가용,영업용,계,관용,자가용,영업용,계
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272,2025-05,전북,전주시 덕진구,158,"151,019","3,074","154,251",103,"3,260",933,...,"2,614","21,650",14,571,403,988,427,"173,734","7,024","181,185"
273,2025-05,전북,전주시 완산구,342,"136,966","2,005","139,313",227,"2,928",539,...,"1,532","17,502",23,359,252,634,803,"156,012","4,328","161,143"
274,2025-05,전북,정읍시,107,"41,491","1,510","43,108",111,"1,490",198,...,895,"15,947",8,254,122,384,388,"58,125","2,725","61,238"
275,2025-05,전북,진안군,110,"9,817",91,"10,018",53,399,95,...,107,"5,272",15,58,16,89,289,"15,328",309,"15,926"


In [17]:
df.iloc[:, 1] + " " + df.iloc[:, 2]

0                                      NaN
1      검색기간  : 202505 ~ 202505 검색분야 : 종합일반
2                                      NaN
3                                  시도명 시군구
4                                  시도명 시군구
                      ...                 
272                             전북 전주시 덕진구
273                             전북 전주시 완산구
274                                 전북 정읍시
275                                 전북 진안군
276                                   전북 계
Length: 277, dtype: object